# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Organizing code without classes”

Namespaces and modules organizes code in F#.

>- Place related types together in namespaces.
>- Place related stateless functions together in modules.

The above implies that:

- namespaces cannot contain functions
- `module` names are essentially `namespace` names
- a `module` can nest another `module`

>… I preferred thinking of modules as static classes. Nowadays, I find it more natural to think of them as namespaces that happen to also be able to store functions. One of the reasons for this is that modules can be _opened_, like namespaces.

The term _opened_ refers to the `open` keyword in F# (like _use_ would refer to the `using` keyword in C#).

## revisiting capstone 1

Let’s revisit the `driveTo` function from [capstone 1](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/08-capstone-01.ipynb), which is composed of three functions and a `type`. We can organize all of these into a `Songhay.Transportation` namespace. We will do this in the most formal .NET-Core way possible (for me) by using the `dotnet` CLI to generate a library and a shell project all under a Solution file (`*.sln`):

In [ ]:
$namespace = "Songhay.Transportation"
$rootFolder = "12-organizing-code-without-classes"

dotnet new console `
    --language F# `
    --name "$($namespace).Shell" `
    --output ./$rootFolder/$namespace/"$($namespace).Shell"

dotnet new classlib `
    --language F# `
    --name $namespace `
    --output ./$rootFolder/$namespace/$namespace

dotnet `
    add ./$rootFolder/$namespace/"$($namespace).Shell/$($namespace).Shell.fsproj" `
    reference ./$rootFolder/$namespace/$namespace/"$($namespace).fsproj"

dotnet new sln `
    --name $namespace `
    --output ./$rootFolder/$namespace

dotnet sln ./$rootFolder/$namespace/"$($namespace).sln" `
    add ./$rootFolder/$namespace/"$($namespace).Shell/$($namespace).Shell.fsproj"

dotnet sln ./$rootFolder/$namespace/"$($namespace).sln" `
    add ./$rootFolder/$namespace/$namespace/"$($namespace).fsproj"

The technical reason why this is needed is because the scripting versions of F# (and C#) do _not_ support `namespace`. So, we need to edit the `Library.fs` file, generated by the script above, to add our `driveTo` logic:

In [ ]:
Get-Content -Path ./12-organizing-code-without-classes/Songhay.Transportation/Songhay.Transportation/Library.fs

namespace Songhay.Transportation

type Destinations =
    | Home = 'h'
    | Office = 'o'
    | Stadium = 's'
    | GasStation = 'g'

module Utility =

    let getDistance (destination) =
        if destination = Destinations.Home then 25
        elif destination = Destinations.Office then 50
        elif destination = Destinations.Stadium then 25
        elif destination = Destinations.GasStation then 10
        else failwith "Unknown destination!"

    let calculateRemainingPetrol (currentPetrol, distance) =
        let remainingPetrol = currentPetrol - distance
        if remainingPetrol > 0 then remainingPetrol
        else failwith "Oops! You’ve run out of petrol!"

    let driveTo (petrol, destination) =
        let distance = getDistance destination
        let remainingPetrol = calculateRemainingPetrol(petrol, distance)
        if destination = Destinations.GasStation then remainingPetrol + 50
        else remainingPetrol


Now, we can `Open` this logic in our shell (`Program.fs`):


In [ ]:
Get-Content -Path ./12-organizing-code-without-classes/Songhay.Transportation/Songhay.Transportation.Shell/Program.fs

// Learn more about F# at http://docs.microsoft.com/dotnet/fsharp

open System

open Songhay.Transportation
open Songhay.Transportation.Utility

[<EntryPoint>]
let main argv =
    let remainingPetrol =
        100
        |> driveTo Destinations.Office
        |> driveTo Destinations.Stadium
        |> driveTo  Destinations.GasStation
        |> driveTo Destinations.Home

    Console.WriteLine $"remaining petrol: {remainingPetrol}"

    0


In [ ]:
dotnet run --project ./12-organizing-code-without-classes/Songhay.Transportation/Songhay.Transportation.Shell/Songhay.Transportation.Shell.fsproj

remaining petrol: 40


@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.246201+da749355d416da20e634e5c80073b92356b57e0eBuild date: 2021-09-12T07:21:44.0000000Zhttps://github.com/dotnet/interactive
